In [1]:
# Insérez ici le numéro de votre équipe
VOTRE_NUMERO_EQUIPE = 4
VOTRE_NUMERO_DINGO = 0
SIM = True

In [3]:
# Import des paquets requis et configuration du ROS Master
import sys
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/')
sys.path.append('/usr/lib/python3/dist-packages')
from os import environ
if SIM:
    environ['ROS_MASTER_URI'] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)
else:
    environ['ROS_MASTER_URI'] = "http://cpr-ets05-0{}:11311/".format(VOTRE_NUMERO_DINGO)
    environ['ROS_IP'] = "192.168.0.81" # adresse IP de votre station de travail
import numpy as np
import rospy
from scipy.spatial.transform import Rotation as R
from math import sin, cos
from jackal_msgs.msg import Feedback
from jackal_msgs.msg import Drive
from sensor_msgs.msg import Imu
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Quaternion, Twist, Vector3
from filterpy.kalman import ExtendedKalmanFilter
#from lab_utils.project2 import publish_odom_tf

In [4]:
# Créer et démarrer un nouveau noeud
rospy.init_node('dingo_state_estimation', anonymous=True)

In [5]:
# Fonction pour le calcul de l'orientation à partir d'un quaternion
# Entrée : Quaternion [x, y ,z ,w]
# Sortie : Angle de lacet (yaw) en radians
def get_heading_from_quaternion(q):
    r = R.from_quat([q.x, q.y, q.z, q.w])
    angles = r.as_euler('xyz', degrees=False)
    return angles[2]

In [6]:
# Ground truth subscriber callback
ground_truth_msg = Odometry()
def ground_truth_callback(msg):
    global ground_truth_msg
    ground_truth_msg = msg

### Configuration du script

Ajoutez ici les données géométriques du robot

In [8]:
# À compléter selon les tâches précédentes
interwheel_distance = 0.46
left_wheel_radius = 0.045
right_wheel_radius = 0.045

### Estimer les déplacements du robot

Dans la prochaine cellule ajoutez votre code pour estimer les variables x, y et theta du robot (et leurs vitesses).

In [10]:
# Encodeurs subscriber callback
# cette fonction se charge aussi de calculer l'odométrie pour mettre à jour l'FK
# l'utilisation de ces variables est suggérée
last_time_ = rospy.Time.now()
def encoders_callback(msg):
    global prev_left_pos, prev_right_pos, last_time_
    global theta_robot_prec, theta_robot, x_robot_prec, y_robot_prec, x_robot, y_robot, vlin_robot, vang_robot, u # Pour que les modifications appliquées à ces variables soient effectives à l'extérieur de la fonction il faut déclarer ses variables comme globales
    
    # Utiliser les valeurs des encodeurs dans cette fonction
    
    # Calcul des vitesses lineraires de chaque roue

    # Obtenir les mesures de chaque encodeur
    left_pos  = msg.drivers[0].measured_travel*right_wheel_radius
    right_pos = msg.drivers[1].measured_travel*left_wheel_radius
    
    # Ajoutez ici votre code pour estimer les déplacements du robot:
    
    # Distance parcourue par chaque roues
    dist_r = right_wheel_radius*(right_pos-prev_right_pos)
    dist_l = left_wheel_radius*(left_pos-prev_left_pos)
    
    # Vitesse de chaque roue 
    vr = dist_r/dt
    vl = dist_l/dt
        
    # Definition vecteur commande
    
    u = np.array([[vr],[vl]])
    
    # Prediction - Filtre de Kalman
    
    kf.predict(u)
    
    # Ajouter z et Timestamp
    #kf_odom_update(z, timestamp) # Optionnel selon le filtre choisi
    
# UMI subscriber callback
# Dans cette fonction la variable msg contient les mesures de l'UMI
def imu_callback(msg):
    global z
    # Utiliser les valeurs du IMU dans cette fonction
    
    acc_x = msg.linear_acceleration.x
    acc_y = msg.linear_acceleration.y
    theta = msg.orientation.z
    omega = msg.angular_velocity.z
    
    # Definition vecteur mesures
    
    z = np.array([[acc_x],
                  [acc_y],
                  [theta],
                  [omega],
                 ])
    
    # Mise a jour - Filtre de Kalman
    kf_pre_update(last_measurement_time)
#     kf.update(z)
    
    # Ajouter z et Timestamp
    #kf_imu_update(z, timestamp) # Optionnel selon le filtre choisi

### Construire le filtre

Dans la prochaine cellule utiliser la bibliothèque FilterPy pour construire le filtre Kalman et estimer le déplacement du robot à partir des capteurs

Vous devez concevoir votre filtre et prendre en compte plusieurs éléments par exemple : nombre d'états, nombre de commandes, nombre de mesures, synchrone ou asynchrone, KF ou EKF, etc.

In [12]:
# Definition de variables
dim_x = 8
dim_z = 4
dim_u = 2

# Création du filtre Kalman - dimensions x et z initialisées (changez le 0 selon votre utilisation)
#kf = KalmanFilter(dim_x=0, dim_z=0)
kf = ExtendedKalmanFilter(dim_x, dim_z, dim_u)

# Fréquence de mise à jour (changez le 1 pour la fréquence des msgs en entrée)
dt = 1/50

# État de départ (changez le 0 selon votre utilisation)
kf.x = np.zeros(dim_x) # On a 8 états

# Matrice de transition d'états
#kf.F = np.array([1, dt^2/2, 0, 0, -(vl+vr)/2*dt*sin()
#                ])

                #c1 c2    c3    c4  c5     c6   c7  c8
kf.F = np.array([[1, 0, (dt^2)/2, 0, 0, 0       , 0, 0], #l1
                 [0, 0, dt      , 0, 0, 0       , 0, 0], #l2
                 [0, 0, 1       , 0, 0, 0       , 0, 0], #l3
                 [0, 0, 0       , 1, 0, (dt^2)/2, 0, 0], #l4
                 [0, 0, 0       , 0, 0, dt      , 0, 0], #l5
                 [0, 0, 0       , 0, 0, 1       , 0, 0], #l6
                 [0, 0, 0       , 0, 0, 0       , 1, 0], #l7
                 [0, 0, 0       , 0, 0, 0       , 0, 0], #l8
                ])

# Matrice de transition des contrôles
kf.B = np.array([[dt*cos(theta_robot)/2  ,         dt*cos(theta_robot)/2],
                 [cos(theta_robot)/2     ,            cos(theta_robot)/2],
                 [0                      ,                             0],
                 [dt*cos(theta_robot)/2  ,         dt*cos(theta_robot)/2],
                 [cos(theta_robot)/2     ,            cos(theta_robot)/2],
                 [0                      ,                             0],
                 [dt/interwheel_distance ,       -dt/interwheel_distance],
                 [1/interwheel_distance  ,        -1/interwheel_distance],
                ])

# Matrice de transition capteurs-états
kf.H = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 1, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 1, 0, 0],
                 [0, 0, 0, 0, 0, 0, 1, 0],
                 [0, 0, 0, 0, 0, 0, 0, 1],
                ])

# Matrice de bruit sur le modèle
# Pour le fonctionnement de la librairie, kf.Q = B*Q*B^T
Q = np.array([[0.001964 ,        0],
              [0        , 0.002072],
             ])
kf.Q = np.dot(np.dot(B,Q), np.transpose(B))

#Matrice de bruit sur les mesures 
kf.R = np.array([[2.436761908318059e-05, 0, 0,  0],
                 [0, 2.4522669777637666e-05, 0, 0],
                 [0, 0, 2.4822889213457126e-05, 0],
                 [0, 0, 0, 2.4238010829617694e-05],
                ])

#Matrice Hx
def Hx(x):
    """ compute measurement for slant range that
    would correspond to state x.
    """
    
    return (x[0]**2 + x[2]**2) ** 0.5    


#Matrice HJacobian
def HJacobian(x):
    
    """ compute Jacobian of H matrix at x """

    horiz_dist = x[0]
    altitude   = x[2]
    denom = sqrt(horiz_dist**2 + altitude**2)
    return array ([[horiz_dist/denom, 0., altitude/denom]]) 
    

# Matrice de bruit sur l'odométrie (selon votre utilisation)
#kf.R_odom = np.array([# entrez ici une matrix de la forme [ligne1],[ligne2]
#                        ])

# Matrice de transition odométrie-états (selon votre utilisation)
#kf.H_odom = np.array([# entrez ici une matrix de la forme [ligne1],[ligne2]
#                        ])

# Matrice de bruit sur l'UMI (selon votre utilisation)
#kf.R_imu = np.array([# entrez ici une matrix de la forme [ligne1],[ligne2]
#                    ])

# Matrice de transition UMI-états (selon votre utilisation)
#kf.H_imu =  np.array([# entrez ici une matrix de la forme [ligne1],[ligne2]
#                    ])

# Enregistrement des etats pour le prochain cycle

prev_left_pos    = left_pos
prev_right_pos   = right_pos
last_time_       = rospy.Time.now()

last_measurement_time = 0
def kf_pre_update(timestamp):
    global last_measurement_time

    if timestamp > last_measurement_time:
        dt = timestamp - last_measurement_time
        last_measurement_time = timestamp

        #v = kf_cmd_vector[0] # Optionnel selon le filtre choisi
        
        # Mettre à jour les matrices qui dépendent du temps si on utilise un EKF
        kf.F = np.array([[1, 0, (dt^2)/2, 0, 0, 0       , 0, 0], #l1
                         [0, 0, dt      , 0, 0, 0       , 0, 0], #l2
                         [0, 0, 1       , 0, 0, 0       , 0, 0], #l3
                         [0, 0, 0       , 1, 0, (dt^2)/2, 0, 0], #l4
                         [0, 0, 0       , 0, 0, dt      , 0, 0], #l5
                         [0, 0, 0       , 0, 0, 1       , 0, 0], #l6
                         [0, 0, 0       , 0, 0, 0       , 1, 0], #l7
                         [0, 0, 0       , 0, 0, 0       , 0, 0], #l8
                        ])
        kf.B = np.array([[dt*cos(theta)/2        ,         dt*cos(theta)/2],
                         [cos(theta)/2           ,            cos(theta)/2],
                         [0                      ,                       0],
                         [dt*cos(theta)/2        ,         dt*cos(theta)/2],
                         [cos(theta)/2           ,            cos(theta)/2],
                         [0                      ,                       0],
                         [dt/interwheel_distance , -dt/interwheel_distance],
                         [1/interwheel_distance  ,  -1/interwheel_distance],
                        ])
        
        predict_update(z, HJacobian, Hx, args=(), hx_args=(), u=dim_u)
        # À remplir au besoin
        
    else:
        pass

    #kf.predict(u=kf_cmd_vector)


def kf_odom_update(z, timestamp): # Optionnel selon le filtre choisi
    kf_pre_update(timestamp)

    kf.H = kf.H_odom

    kf.R = kf.R_odom

    kf.update(z)

def kf_imu_update(z, timestamp): # Optionnel selon le filtre choisi
    kf_pre_update(timestamp)

    kf.H = kf.H_imu

    kf.R = kf.R_imu

    kf.update(z)

NameError: name 'ExtendedKalmanFilter' is not defined

### Configuration du script

Ajoutez ici les noms des messages ROS

In [ ]:
# ROS subscribers et publishers
feedback_sub = rospy.Subscriber('mobile_manip/dingo_velocity_controller/feedback', Feedback, encoders_callback)
ground_truth_sub = rospy.Subscriber('mobile_manip/t265/odom/sample', Odometry, ground_truth_callback)

if SIM:
    imu_sub = rospy.Subscriber('imu/data', Imu, imu_callback)
    cmd_drive_pub = rospy.Publisher('mobile_manip/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)
else:
    imu_sub = rospy.Subscriber('mobile_manip/base/imu/data', Imu, imu_callback)
    cmd_drive_pub = rospy.Publisher('mobile_manip/base/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)

### Tester le filtre

La prochaine cellule déplace le robot dans une courbe pour 20 secondes et enregistre sa pose réelle la pose estimée dans un rosbag.

Changez les 0 pour les valeurs correspondantes de votre filtre Kalman.

In [ ]:
# Création d'un nouveau contenant ROSBAG (en écriture)
import rosbag
test_bag = rosbag.Bag('project2.bag', 'w')

# Publie l'odométrie et la TF pendant 20 secondes
start = float(rospy.Time().now().secs)
rate = rospy.Rate(50) # 50hz
while (float(rospy.Time().now().secs) - start) < 20:
    
    # ***TÂCHE***
    # Remplissez avec les valeurs correspondantes de votre filtre Kalman (états)
    # exemple x = kf.x[0]
    x = kf.x[0]
    y = kf.x[3]
    vlin = (kf.x[1]**2+kf.x[4]**2)**0.5
    vang = kf.x[7]
    theta = kf.x[6]
    # ***TÂCHE***

    # Déplacer le robot
    cmd_drive_msg = Drive()
    cmd_drive_msg.drivers[0] = 5.0
    cmd_drive_msg.drivers[1] = 7.0
    cmd_drive_pub.publish(cmd_drive_msg)
    
    # Enregistrer le déplacement du robot
    pose = (x, y, 0)
    r = R.from_euler('xyz', [0, 0, theta], degrees=False)
    orientation = r.as_quat()
    odometry_msg = Odometry()
    odometry_msg.header.frame_id = "odom"
    odometry_msg.header.stamp = rospy.Time.now()
    odometry_msg.child_frame_id = "base_link"
    odometry_msg.pose.pose = Pose(Point(*pose), Quaternion(*orientation))
    odometry_msg.twist.twist = Twist(Vector3(vlin, 0, 0), Vector3(0, 0, vang))
    
    test_bag.write('/filter', odometry_msg, rospy.Time().now())
    test_bag.write('/ground_truth', ground_truth_msg, rospy.Time().now())
    rate.sleep()
    
# fermer proprement le ROSBAG
test_bag.close()

In [ ]:
# Arreter le dingo
cmd_drive_msg = Drive()
cmd_drive_msg.drivers[0] = 0.0
cmd_drive_msg.drivers[1] = 0.0
cmd_drive_pub.publish(cmd_drive_msg)